In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "mps")

In [63]:
from data_proccessing import labels

# Define DenseNet model
dense = models.densenet121()

# dense.features[0] = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # Modify the first layer to accept 3 channels
dense.classifier = nn.Linear(in_features=1024, out_features=len(labels))  # Modify the last layer for multi-label classification
# dense.to(device)

for param in dense.parameters():
    param.requiresGrad = False
    
for param in dense.classifier.parameters():
    param.requires_grad = True


print(dense)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [64]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class DenseNet(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, growth_rate, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels + growth_rate, growth_rate, kernel_size=3, stride=1, padding=1)
    
    def forward(self, x):
        out1 = F.relu(self.conv1(x))
        out2 = F.relu(self.conv2(torch.cat([x, out1], 1)))
        out = torch.cat([x, out1, out2], 1)  # Concatenate along the channels dimension
        return out

# Testing the model
if __name__ == "__main__":
    # Define a sample input tensor with the provided image shape
    sample_input = torch.randn(32, 3, 1024, 1024)
    model = DenseNet(in_channels=3, growth_rate=12)
    output = model(sample_input)
    print("Output shape:", output.shape)

Output shape: torch.Size([32, 27, 1024, 1024])


In [66]:
from data_proccessing import train_loader, valid_loader,train_loader, labels
# # Define loss function
criterion = nn.BCELoss()
# criterion = nn.CrossEntropyLoss()


# # Define optimizer
optimizer = optim.Adam(dense.parameters(), lr=0.001)

dense.train()
dense.to(device)
# Retrain
num_epochs = 3
for epoch in range(num_epochs):
    for i, (train_data) in enumerate(train_loader):
        # Forward pass
        image = train_data[0]
        output = train_data[1]
        
        print(image.shape)
        
        outputs = dense(image.to(device))
        loss = criterion(outputs, labels.to(device))
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Display
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                                      num_epochs, \
                                                                      i+1, \
                                                                      len(train_loader), \
                                                                      loss.item()))

torch.Size([32, 3, 1024, 1024])


RuntimeError: MPS backend out of memory (MPS allocated: 16.93 GB, other allocations: 704.00 KB, max allowed: 18.13 GB). Tried to allocate 2.00 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [11]:
class _DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(_DenseLayer, self).__init__()
        self.norm1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, growth_rate*4, kernel_size=1, stride=1, bias=False)
        self.norm2 = nn.BatchNorm2d(growth_rate*4)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(growth_rate*4, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(self.relu1(self.norm1(x)))
        out = self.conv2(self.relu2(self.norm2(out)))
        out = torch.cat([x, out], 1)
        return out
    



In [12]:
class _DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super(_DenseBlock, self).__init__()
        self.layers = nn.ModuleList([_DenseLayer(in_channels + i * growth_rate, growth_rate) for i in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [13]:
class _Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(_Transition, self).__init__()
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        out = self.pool(self.conv(self.relu(self.norm(x))))
        return out

In [47]:
import torch.nn.functional as F
from data_proccessing import image_size
class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv0 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False).to(torch.float32)
        self.norm0 = nn.BatchNorm2d(64)
        self.relu0 = nn.ReLU(inplace=True)
        self.pool0 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.denseblock1 = _DenseBlock(num_layers=3, in_channels=64, growth_rate=4)  # Reduced growth rate
        self.transition1 = _Transition(in_channels=256, out_channels=128)
        self.denseblock2 = _DenseBlock(num_layers=3, in_channels=128, growth_rate=4)  # Added second dense block
        self.norm5 = nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)  # Adjusted to the output size of the last dense block
        self.classifier = nn.Linear(in_features=64*image_size*image_size, out_features=15, bias=True)  # Adjusted input features

    def forward(self, x):
        x = self.conv0(x.to(torch.float32))
        x = self.norm0(x)
        x = self.relu0(x)
        out = self.pool0(x)
        out = self.denseblock1(out)
        out = self.transition1(out)
        out = self.denseblock2(out)
        out = F.relu(self.norm5(out))
        out = F.avg_pool2d(out, kernel_size=out.size(2))
        out = out.view(out.size(0), -1)
        out = self.classifier(x)
        return out
    

    
model = DenseNet()
print(model)

DenseNet(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (classifier): Linear(in_features=3211264, out_features=15, bias=True)
)


In [48]:
from data_proccessing import train_loader,train_loader, labels

model.train()
model.to(device)
# Retrain
num_epochs = 3
for epoch in range(num_epochs):
    for i, (train_data) in enumerate(train_loader):
        # Forward pass
        image = train_data[0]
        output = train_data[1]
        outputs = model(image)
        loss = criterion(outputs, labels.to(device))
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Display
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                                      num_epochs, \
                                                                      i+1, \
                                                                      len(train_loader), \
                                                                      loss.item()))

RuntimeError: Mismatched Tensor types in NNPack convolutionOutput